In [16]:
#Code for solving the subproblem
#Iteratively increasing the levels of generated list and apply domination algorithm after each level increases 
import pickle
import pandas as pd
import numpy as np
import datetime as time
%run ExcelToObjects.ipynb
compat = pd.read_excel("Input Files/Sample 6/aircraft_airport_compatibility.xlsx")

1:30
157
157
This is flight number-3514, aircraft=VTICD, Start-DEL, End-CJB, Dep-7:40, arrival-11:0, flight duration=200 minutes
Length of dict keys: 157
This is flight number-3638, aircraft=VTICX, Start-MAA, End-DEL, Dep-21:30, arrival-24:20, flight duration=170 minutes
This is flight number-3514, aircraft=VTICD, Start-DEL, End-CJB, Dep-7:40, arrival-11:0, flight duration=200 minutes
44
{'VTICD': [3514, 3515, 3516, 3517], 'VTICX': [3638], 'VTICF': [3755, 3756, 3757, 3758], 'VTICG': [3888, 3889, 3890], 'VTICH': [4021, 4022, 4023, 4024, 4025], 'VTICM': [4156, 4157, 4158, 4159], 'VTICN': [4289], 'VTICO': [4417, 4418, 4419, 4420], 'VTICB': [4535, 4536], 'VTICQ': [4657, 4658, 4659, 4660, 4661], 'VTXDD': [4765], 'VTXDX': [4825, 4826, 4827, 4828], 'VTXES': [4912, 4913, 4914, 4915, 4916], 'VTXEP': [5021, 5022, 5023, 5024], 'VTXEX': [5358, 5359], 'VTXEF': [5481, 5482, 5483], 'VTXEG': [5605, 5606, 5607, 5608], 'VTXEH': [5724, 5725, 5726, 5727], 'VTXEC': [5849, 5850, 5851, 5852], 'VTXEJ': [5970,

KeyError: 7766

In [17]:
#Importing the relevant data for the flights, maintenance, etc.
file1 = open("Preprocessed Objects/Sample 1/flight_dict", 'rb')
file2 = open("Preprocessed Objects/Sample 1/maint_dict", 'rb')
file3 = open("Preprocessed Objects/Sample 1/notam_dict", 'rb')
file4 = open("Preprocessed Objects/Sample 1/start_dict", 'rb')
file5 = open("Preprocessed Objects/Sample 1/penalty_dict", 'rb')
flight_obj_dict = pickle.load(file1)
maint_obj_dict = pickle.load(file2)
notam_obj_dict = pickle.load(file3)
start_obj_dict = pickle.load(file4)
penalty_obj_dict = pickle.load(file5)

In [18]:
#make a class of LOF related data
class lof:
    'A class of route with attributes flights and available time after the end flight'
    def __init__(self, flights, avail_time_end, port_end, cost, inf):
        self.flights= flights
        self.avail_time_end=avail_time_end
        self.port_end=port_end
        self.cost=cost
        self.red_cost=0
        self.info = inf
        
    def extend(self, F_added):
        L=[]            #defining a list of LOFs
        for f in F_added:
            l=lof(self.flights+[f[0]], f[1], f[2])
            L.append(l)
        return L
    
    def display(self):
        print(self.info)
        '''
        print("LOF")
        for f in self.flights:
            print(" %d"%f.num , end=" ")
        print("\n avail time in hours= %f"%(self.avail_time_end/60))
        print("End port="+str(self.port_end))
        print("Cost=$"+str(self.cost))
        '''
        
    def is_f(self, f):
        for i in self.flights:
            if i!='m' and f.num == i.num:
                return 1
        return 0
    
    def is_p(self, p):
        if p==self.port_end:
            return 1
        else:
            return 0
        
    def calc_red(self,dual1, dual2_, dual3):
        dual_sum=0
        for f in self.flights:
            if f!= 'm':
                dual_sum = dual_sum + dual1[f.num]
        dual_sum = dual_sum + dual2_
        dual_sum = dual_sum + dual3[self.port_end]
        self.red_cost = self.cost - dual_sum
        return self.red_cost

In [19]:
airport=[]
for i in flight_obj_dict.keys():
    if flight_obj_dict[i].orig not in airport:
        airport.append(flight_obj_dict[i].orig)
    if flight_obj_dict[i].dest not in airport:
        airport.append(flight_obj_dict[i].dest)

In [20]:
#Solving the subproblem
#Checks extensions of LOF conditions
def check_conditions(f, airplane_avail_time, avail_port, c=None):
    f1=f2=f3=f4=0
    f2=1
    #Condition 1 : Night curfew
    if f.dur+airplane_avail_time<26*60:
        f1=1
        '''
    if c is not None:
        if compat[c][f.dest]!=1:
            f2=0
            '''
    if f1==1 and f2==1:
        return 1
    else:
        return 0

#creates a start list of lofs for a particular aircraft given its start time and start airport+
def add_s(startflights, airplane_avail_time, avail_port, c=None): 
    L=[]
    for f in startflights[avail_port]:
        if  airplane_avail_time-(f.dep)>=0:
            if check_conditions(f, airplane_avail_time, avail_port):
                L.append(lof([f], f.dur+airplane_avail_time+30, f.dest, 20*(airplane_avail_time-(f.dep)),\
               str(f.num)+" "+ f.orig+" "+f.dest+ " "\
                +hour_min(airplane_avail_time)+" "+\
                hour_min(f.dur+airplane_avail_time)))
        else:
            if check_conditions(f, airplane_avail_time, avail_port):
                L.append(lof([f], f.dur+f.dep, f.dest, 0,\
               str(f.num)+" "+ f.orig+" "+f.dest+ " "\
                +hour_min(f.dep)+" "+\
                hour_min(f.dur+f.dep)))
    return L

In [21]:
def create_next_l(l , f, notam_obj_dict, mode, dual1, dual2_,dual3):
    #Create next level lof by adding f in the lof l. Two modes of the function 
    if mode==1:
        max_time = 0
        if l.port_end in list(notam_obj_dict.keys()):
            if l.avail_time_end>=notam_obj_dict[l.port_end].start and l.avail_time_end<=notam_obj_dict[l.port_end].end:
                max_time = notam_obj_dict[l.port_end].end + f.dur + 30
            else:
                max_time = l.avail_time_end+f.dur + 30 
        if f.dest in list(notam_obj_dict.keys()):
            if l.avail_time_end+f.dur>=notam_obj_dict[f.dest].start and l.avail_time_end+f.dur<=notam_obj_dict[f.dest].end:
                t_av = notam_obj_dict[f.dest].end + 30
                if t_av>max_time:
                    max_time=t_av
            else:
                max_time = max(l.avail_time_end+f.dur + 30, max_time)
        if l.port_end not in list(notam_obj_dict.keys()) and f.dest not in list(notam_obj_dict.keys()):
            max_time = l.avail_time_end+f.dur + 30 
        new_l=lof(l.flights+[f], max_time, f.dest, l.cost+20*(max_time-f.arr-30),\
               l.info + "\n"+str(f.num)+" "+ f.orig+" "+f.dest+ " "\
                +hour_min(max_time - f.dur - 30)+" "+\
                hour_min(max_time-30))       
                
    elif mode==2:
        max_time = 0
        if f.orig in list(notam_obj_dict.keys()):
            if f.dep>=notam_obj_dict[f.orig].start and f.dep<=notam_obj_dict[f.orig].end:
                max_time = notam_obj_dict[f.orig].end + f.dur + 30
            else:
                max_time = f.arr.hour*60 + f.arr.minute + 30 
        if f.dest in list(notam_obj_dict.keys()):
            if f.dep.hour+f.dur>=notam_obj_dict[f.dest].start and f.dep.hour+f.dur<=notam_obj_dict[f.dest].end:
                t_av = notam_obj_dict[f.dest].end + 30
                if t_av>max_time:
                    max_time=t_av
            else:
                max_time = max(f.arr + 30,max_time)
        if f.orig not in list(notam_obj_dict.keys()) and f.dest not in list(notam_obj_dict.keys()):
            max_time = f.arr + 30 
        new_l=lof(l.flights+[f], max_time, f.dest, l.cost+20*(max_time-f.arr-30),\
               l.info + "\n"+str(f.num)+" "+ f.orig+" "+f.dest+ " "\
                +hour_min(max_time - f.dur - 30)+" "+\
                hour_min(max_time-30))      
    if max_time>26*60:
        return -1
    
    new_l.calc_red(dual1, dual2_, dual3)
    if l.avail_time_end-30 == new_l.avail_time_end- f.dur - 30:
        print("Error in flight ",f.num)
    return new_l

In [25]:
# grows next layer into the list of LOF
def add_nex(L, startflights, notam_obj_dict, dual1, dual2_, dual3, c=None):                
    next_L=L[:]
    flag=0
    for l in L:
        flag_l=0
        if c in list(maint_obj_dict.keys()) and l.avail_time_end>maint_obj_dict[c].start and 'm' not in l.flights:
            new_l = lof(l.flights+['m'], l.avail_time_end + maint_obj_dict[c].end-maint_obj_dict[c].start,l.port_end, l.cost,\
                       l.info+"\n Maintenance:"+str(maint_obj_dict[c].end-maint_obj_dict[c].start)+" min")
            next_L.append(new_l)
            next_L.remove(l)
            flag = 1
        else:
            for f in startflights[l.port_end]:
                #Start here : if l.avail_time is within 2 hour early of maintent deadline or exceeds then maintain the aircraft
                if f.dur+l.avail_time_end<26*60 and l.avail_time_end-(f.dep)>=0 and l.is_f(f)==0 :
                #and compat[c][f.dest]==1:
                    if create_next_l(l , f, notam_obj_dict, 1, dual1, dual2_,dual3)!=-1:
                        flag_l=1
                        flag=1
                        new_l = create_next_l(l , f, notam_obj_dict, 1, dual1, dual2_,dual3)
                        next_L.append(new_l)
                elif f.dur+f.dep<26*60 and l.avail_time_end-(f.dep)<=0 and l.is_f(f)==0 :
                #compat[c][f.dest]==1:
                    if create_next_l(l , f, notam_obj_dict, 2, dual1, dual2_,dual3)!=-1:
                        flag_l=1
                        flag=1
                        new_l = create_next_l(l , f, notam_obj_dict, 2, dual1, dual2_,dual3)
                        next_L.append(new_l)
            if flag_l==1:
                next_L.remove(l)
    #next_L[-1].display()
    L[:]=next_L
    #L[-1].display()
    return flag

In [23]:
#Domination step: Delete the elements with greater cost ending at the same airport
def domination(L):
    non_dom = []
    port_end=[]
    for l in L:
        if l.port_end not in port_end:
            port_end.append(l.port_end)
    for p in port_end:
        minCost = 10000000
        for l in L:
            if l.port_end==p and l.red_cost<minCost:
                minCost = l.red_cost
                l_best = l
        non_dom.append(l_best)
    '''
    print("Domination procedure")
    print("---")
    print("###Set L###")
    for l in L:
        l.display()
    print("###Set non_dom###")
    for l in non_dom:
        l.display()
    print("---")
    '''
    return non_dom

In [24]:
def subprob(dual1, dual2, dual3):
    Pos_lof_c={i:[] for i in list(start_obj_dict.keys())}
    for i in Pos_lof_c.keys():
        Pos_lof_c[i]=add_s(startflights, start_obj_dict[i].start_time, start_obj_dict[i].start_airport, i)
        dual2_ = dual2[i]
        flag=1
        while flag==1:
            flag=add_nex(Pos_lof_c[i], startflights, notam_obj_dict, dual1, dual2_,dual3)
            Pos_lof_c[i] = domination(Pos_lof_c[i])
    Pos_lof_c2={i:[] for i in list(start_obj_dict.keys())}
    for i in Pos_lof_c.keys():     
        min_red = 100000
        for l in Pos_lof_c[i]:
            if l.red_cost<min_red:
                min_red=l.red_cost
                l_this = l
        Pos_lof_c2[i] = [l_this]
        '''
    for i in Pos_lof_c.keys():
        print("---",i,"---")
        Pos_lof_c2[i][0].display()'''
    return Pos_lof_c2
    

In [14]:
dual1={}
dual2={}
dual3={}
for f in flight_obj_dict.keys():
    dual1[f]=100
for c in start_obj_dict.keys():
    dual2[c] = 20
for p in ['SIN', 'BOM', 'MAA', 'AUH', 'TRZ', 'COK', 'MCT']:
    dual3[p] = 0
dual3['COK'] = 100
dual3['AUH'] = 100
dual3['MAA'] = 100

In [15]:
dual1 = {7766: 2000.0, 7800: 2300.0, 7834: 9600.0, 18461: -1100.0, 4919: 0, 8139: 0, 8174: 0, 8680: 1000.0, 8972: 2900.0, 16300: 1500.0, 16334: 0, 6831: 0, 6971: 0, 7629: -35300.0, 7690: 28800.0, 9453: 4300.0, 9488: 0, 11472: 1300.0, 11473: 8000.0, 16862: 0, 16863: 0, 6889: -1500.0, 6890: 0, 6944: 0, 15217: 0, 15236: 0, 4451: 0, 18255: 0, 18320: 0, 4230: -2200.0, 14439: 0, 14640: 2200.0, 18106: 0, 6297: 0, 6355: 0, 8701: 0, 8720: 0, 14674: 0, 17225: -2100.0, 13643: 0, 13653: 0, 16396: 0, 16406: 0, 4196: 2900.0, 10636: 0, 10797: 7000.0, 15537: 0, 16176: 6600.0, 17020: 0, 17668: -36200.0, 17702: 28800.0, 19756: 0, 19791: 0, 4383: 0, 4418: 0, 10269: 0, 10317: 0, 10318: 0, 20187: 0, 20290: 0, 21037: 0, 22604: -24500.0, 22638: 28800.0, 22739: 0, 22773: 0, 20903: -28800.0, 20914: 28800.0, 21240: 0, 20558: 0, 22302: 0, 20700: 0, 20736: 0, 20771: 0, 20629: 0, 20643: 0, 22501: 0, 22107: 0, 22108: 0, 22145: 0, 22146: 0, 20109: 0, 20128: 8400.0, 22689: 0, 22708: 0, 20256: 0, 20925: 0, 21066: 0, 21636: 0, 21729: 0, 21763: 0, 21809: 0, 22884: 0, 22234: 0, 22268: 0, 22371: 0, 22405: 0, 23111: 0, 23152: 0, 23168: 0, 20363: 28800.0, 20397: 28800.0, 22569: 28800.0, 5426: 2100.0, 8002: 11000.0, 5440: 0, 5459: 0, 7221: 0, 17874: 0, 5909: 0, 6191: 0, 5187: 0, 7871: -2300.0, 7872: 0, 7921: 0, 6273: 0, 7183: 0, 6041: 0, 7253: 0, 7398: 0, 7399: 0, 7400: 0, 7490: -27400.0, 7526: 28800.0, 7202: 0, 19509: 0, 5380: 28800.0, 3958: 0, 5252: 700.0, 5253: 2100.0, 9519: 2300.0, 5131: 0, 5484: 0, 5553: 0, 4993: 0, 18208: -28800.0, 18230: 28800.0, 6444: 0, 5483: 0, 5554: 0, 3584: -28700.0, 3620: 28800.0, 3657: 0, 3720: 0} 

dual2 = {'VTEDD': -6000.0, 'VTCIG': 0, 'VTEXJ': 0, 'VTEXH': -3700.0, 'VTEXI': 0, 'VTEXG': 0, 'VTCIQ': -9000.0, 'VTCIO': -10500.0, 'VTCIN': 0, 'VTCIM': -8400.0, 'VTCIH': 0, 'VTCIF': -3100.0, 'VTCID': -10500.0, 'VTAYB': 0, 'VTAYA': -4300.0, 'VTAXZ': 0, 'VTAXX': 0, 'VTAXW': 0, 'VTAXU': 0, 'VTAXT': 0, 'VTAXR': -8400.0, 'VTAXP': 0, 'VTAXN': 0, 'VTAXM': 0, 'VTAXJ': 0, 'VTAXI': 0, 'VTAXH': -86400.0, 'VTANY': -13100.0, 'VTANX': 0, 'VTANV': 0, 'VTANU': 0, 'VTANT': 0, 'VTANR': -10500.0, 'VTANQ': 0, 'VTANP': -8200.0, 'VTANN': 0, 'VTANL': 0, 'VTANK': 0, 'VTANI': 0, 'VTANH': 0, 'VTANG': 0, 'VTANE': 0, 'VTAND': -10500.0, 'VTANC': -5100.0, 'VTALN': 0, 'VTALL': -10500.0, 'VTALK': 0, 'VTALJ': 0, 'VTALH': 0, 'VTALF': -10500.0, 'VTAIY': 0} 

dual3 = {'SIN': 0, 'BOM': 0, 'MAA': 0, 'AUH': 0, 'DEL': 10500.0, 'PAT': 0, 'LKO': 0, 'TRV': 0, 'MLE': 0, 'BLR': 0, 'BKK': -21900.0, 'VGA': 0, 'HYD': 0, 'BBI': 0, 'COK': 0, 'DXB': 0, 'GOI': 0, 'RUH': 0, 'ATQ': 0, 'NDC': 0, 'CCU': 0, 'PNQ': 0, 'GAU': 0, 'SXR': 0, 'IXZ': 0, 'VTZ': 0, 'CCJ': 0, 'TRZ': 0, 'AAN': 0, 'SHJ': 0, 'IXM': 0, 'CJB': 0, 'MCT': 0, 'CNN': 0, 'IXE': 0, 'VNS': 0, 'FCO': 0, 'FRA': -28800.0, 'MEL': 0, 'CDG': 0, 'LHR': 0, 'PVG': 0, 'AMD': 0, 'NRT': 0, 'VIE': 0, 'HKG': 0, 'ICN': 0, 'BHX': 0, 'ORD': 0, 'ARN': 0, 'JFK': 0, 'JED': 0, 'SFO': 0, 'SAG': 0, 'ISK': 0}

PL = subprob(dual1, dual2, dual3)

KeyError: 'VTICD'

In [18]:
start_obj_dict

{'VTEDD': <__main__.aircraft at 0x1c5c59c7a58>,
 'VTCIG': <__main__.aircraft at 0x1c5c59c77f0>,
 'VTEXJ': <__main__.aircraft at 0x1c5c59c78d0>,
 'VTEXH': <__main__.aircraft at 0x1c5c59c7630>,
 'VTEXI': <__main__.aircraft at 0x1c5c59c7550>,
 'VTEXG': <__main__.aircraft at 0x1c5c59c74e0>,
 'VTCIQ': <__main__.aircraft at 0x1c5c59c7470>,
 'VTCIO': <__main__.aircraft at 0x1c5c59c7400>,
 'VTCIN': <__main__.aircraft at 0x1c5c59c7320>,
 'VTCIM': <__main__.aircraft at 0x1c5c59c7240>,
 'VTCIH': <__main__.aircraft at 0x1c5c59c7208>,
 'VTCIF': <__main__.aircraft at 0x1c5c59c7128>,
 'VTCID': <__main__.aircraft at 0x1c5c59c7080>,
 'VTAYB': <__main__.aircraft at 0x1c5c5374390>,
 'VTAYA': <__main__.aircraft at 0x1c5c59fc940>,
 'VTAXZ': <__main__.aircraft at 0x1c5c59fcb70>,
 'VTAXX': <__main__.aircraft at 0x1c5c59fcd68>,
 'VTAXW': <__main__.aircraft at 0x1c5c59f02b0>,
 'VTAXU': <__main__.aircraft at 0x1c5c59f0a20>,
 'VTAXT': <__main__.aircraft at 0x1c5c59f0d68>,
 'VTAXR': <__main__.aircraft at 0x1c5c59

In [19]:
flight_obj_dict

{7766: <__main__.flight at 0x1c5c5809048>,
 7800: <__main__.flight at 0x1c5c56fb358>,
 7834: <__main__.flight at 0x1c5c56fbc88>,
 18461: <__main__.flight at 0x1c5c56fb6a0>,
 4919: <__main__.flight at 0x1c5c56fb518>,
 8139: <__main__.flight at 0x1c5c56fb550>,
 8174: <__main__.flight at 0x1c5c56fbeb8>,
 8680: <__main__.flight at 0x1c5c56fbef0>,
 8972: <__main__.flight at 0x1c5c56fbe80>,
 16300: <__main__.flight at 0x1c5c56fb2b0>,
 16334: <__main__.flight at 0x1c5c56e6c18>,
 6831: <__main__.flight at 0x1c5c56e6eb8>,
 6971: <__main__.flight at 0x1c5c56e68d0>,
 7629: <__main__.flight at 0x1c5c5b42630>,
 7690: <__main__.flight at 0x1c5c5b42c50>,
 9453: <__main__.flight at 0x1c5c5b42908>,
 9488: <__main__.flight at 0x1c5c5b42898>,
 11472: <__main__.flight at 0x1c5c5b428d0>,
 11473: <__main__.flight at 0x1c5c5b42358>,
 16862: <__main__.flight at 0x1c5c5b423c8>,
 16863: <__main__.flight at 0x1c5c5b42668>,
 6889: <__main__.flight at 0x1c5c5b42710>,
 6890: <__main__.flight at 0x1c5c5b42390>,
 694

In [20]:

airport

['SIN',
 'BOM',
 'MAA',
 'AUH',
 'DEL',
 'PAT',
 'LKO',
 'TRV',
 'MLE',
 'BLR',
 'BKK',
 'VGA',
 'HYD',
 'BBI',
 'COK',
 'DXB',
 'GOI',
 'RUH',
 'ATQ',
 'NDC',
 'CCU',
 'PNQ',
 'GAU',
 'SXR',
 'IXZ',
 'VTZ',
 'CCJ',
 'TRZ',
 'AAN',
 'SHJ',
 'IXM',
 'CJB',
 'MCT',
 'CNN',
 'IXE',
 'VNS',
 'FCO',
 'FRA',
 'MEL',
 'CDG',
 'LHR',
 'PVG',
 'AMD',
 'NRT',
 'VIE',
 'HKG',
 'ICN',
 'BHX',
 'ORD',
 'ARN',
 'JFK',
 'JED',
 'SFO',
 'SAG',
 'ISK']

In [21]:
subprob(dual1, dual2, dual3)

KeyError: 'DEL'

In [22]:
fls = [21763 , 21729,  21636  ,21809] 
for f in fls:
    flight_obj_dict[f].display()

This is flight number-21763, aircraft=VTAXN, Start-COK, End-MCT, Dep-02:35:00, arrival-06:05:00, flight duration=210 minutes
This is flight number-21729, aircraft=VTAXN, Start-MCT, End-COK, Dep-07:20:00, arrival-11:00:00, flight duration=220 minutes
This is flight number-21636, aircraft=VTAXN, Start-COK, End-AUH, Dep-11:50:00, arrival-15:50:00, flight duration=240 minutes
This is flight number-21809, aircraft=VTAXN, Start-AUH, End-COK, Dep-16:50:00, arrival-20:55:00, flight duration=245 minutes


In [79]:
flight_obj_dict[21809].display()

This is flight number-21809, aircraft=VTAXN, Start-AUH, End-COK, Dep-16:50:00, arrival-20:55:00, flight duration=245 minutes


In [81]:
notam_obj_dict['AUH'].display()

This is NOTAM number-1, airport-AUH, start-780, end-1020


In [96]:
start_obj_dict['VTAXN'].display()

This is aicraft number-1, model type-VTAXN, start-90, start station-COK


In [90]:
start_obj_dict.keys()

dict_keys(['VTAXN', 'VTEXJ', 'VTAYA'])